In [1]:
from ipsl_dcpp.model.ipsl_dataset import IPSL_DCPP
import torch
import lightning as pl
from ipsl_dcpp.model.pangu import PanguWeather
from hydra import compose, initialize
from omegaconf import OmegaConf

import hydra
import os
os.environ['SLURM_NTASKS_PER_NODE'] = '1'
#torch.set_default_dtype(torch.float32)
# os.environ["CUDA_VISIBLE_DEVICES"]=""
#torch.set_default_tensor_type(torch.FloatTensor)

with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config")
pl.seed_everything(cfg.experiment.seed)
train = hydra.utils.instantiate(
    cfg.experiment.train_dataset,
    generate_statistics=False,
    surface_variables=cfg.experiment.surface_variables,
    depth_variables=cfg.experiment.depth_variables,
    plev_variables=cfg.experiment.plev_variables,
    normalization='climatology',
    delta=False,
    work_path=cfg.environment.work_path,
    scratch_path=cfg.environment.scratch_path,
)

train_dataloader = torch.utils.data.DataLoader(
    train,
    batch_size=1,
    shuffle=True,
    num_workers=1
)

#batch = next(iter(train_dataloader))
model = hydra.utils.instantiate(
    cfg.experiment.module,
    backbone=hydra.utils.instantiate(
        cfg.experiment.backbone,
    ),
    dataset=train_dataloader.dataset

)
trainer = pl.Trainer(
    max_epochs=cfg.experiment.max_epochs,
    enable_checkpointing=True,
    log_every_n_steps=1,
   # max_steps=cfg.experiment.max_steps if not cfg.debug else 10,
    precision="16-mixed",
    #precision='32',
    profiler='simple' if cfg.debug else None,
   # devices=cfg.experiment.num_gpus,
   # strategy='ddp_find_unused_parameters_true',
    #limit_train_batches=0.01 if cfg.debug else 1
    #limit_val_batches=0.01 if cfg.debug else 1,
    num_sanity_val_steps=0,
  #  device='cpu',
  #accelerator='mps',
  #CONV3D not supported by mps, have to use cpu when local 
    accelerator= 'mps' if cfg.environment.name == 'local' else 'gpu',
    fast_dev_run=20
)
trainer.fit(
    model=model,
    train_dataloaders=train_dataloader)
trainer.logged_metrics

Seed set to 0
/Users/gclyne/miniforge3/envs/env_dcpp/lib/python3.12/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
/Users/gclyne/miniforge3/envs/env_dcpp/lib/python3.12/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1708025535429/work/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


91


Using 16bit Automatic Mixed Precision (AMP)
/Users/gclyne/miniforge3/envs/env_dcpp/lib/python3.12/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/gclyne/miniforge3/envs/env_dcpp/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Running in `fast_dev_run` mode: will run th

Training: |          | 0/? [00:00<?, ?it/s]

/Users/gclyne/miniforge3/envs/env_dcpp/lib/python3.12/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


torch.Size([1, 192, 1, 72, 72]) after unsqueeze,and embedding
after torch.Size([1, 91, 143, 144])
tensor([ 2.5272e-03,  1.7333e-02, -1.1267e-02, -1.9381e-02,  2.0015e-02,
         2.7307e-02,  1.4501e-02,  7.0519e-03, -5.4958e-03, -1.2242e-03,
         2.1891e-02,  3.5490e-02,  2.6684e-02,  3.4009e-03, -1.6905e-02,
        -2.1993e-03,  2.3205e-02,  1.4835e-02,  8.3156e-03,  1.6487e-02,
         2.5331e-02,  5.9835e-02,  7.6676e-02,  2.4506e-02, -1.6197e-02,
        -7.9183e-03,  2.5672e-02,  5.0360e-02,  4.7390e-02,  1.3290e-02,
        -4.1632e-03,  1.1778e-02,  2.5821e-02,  2.0806e-02,  1.9845e-02,
         3.2498e-02,  2.6215e-02,  2.5390e-02,  1.6223e-02,  8.2349e-03,
         2.9854e-02,  2.8480e-02,  1.1085e-02,  1.8669e-02,  2.3635e-02,
         1.7731e-02,  1.4228e-02,  2.0133e-02,  2.0408e-02,  5.2895e-03,
        -1.1656e-02, -4.7515e-03,  1.9562e-02,  3.8259e-02,  3.8326e-02,
         2.5768e-02,  2.7669e-02,  3.1316e-02,  1.9556e-02, -9.7020e-04,
        -4.0623e-03,  2.00

`Trainer.fit` stopped: `max_steps=20` reached.


{'train_loss': tensor(0.0459)}

In [2]:
import torch
x = torch.Tensor([[1,2,3],[1,2,4]])

In [4]:
x.sum(1)

tensor([6., 7.])